***LC_net***

In [7]:
import torch
import torch.nn as nn
from accelerate import Accelerator
from torch.optim.optimizer import Optimizer, required
from torchvision.models import resnet50
from transformers import ViTModel, ViTConfig
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, ToTensor, Normalize, RandomHorizontalFlip, RandomResizedCrop, v2, Resize
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from accelerate import notebook_launcher
import os
import timm
import time

os.environ["CUDA_VISIBLE_DEVICES"] = '0'


class lcnet_ft(nn.Module):
    def __init__(self, num_classes=10, embed_dim=1280):
        super(lcnet_ft, self).__init__()

        self.embed_dim = embed_dim
        
        # Load and freeze lcnet
        self.encoder = timm.create_model('timm/lcnet_075.ra2_in1k', pretrained=True, num_classes=num_classes)
        # for param in self.encoder.parameters():
        #     param.requires_grad = False

        # unfreeze the last layer
        # for param in self.encoder.classifier.parameters():
        #     param.requires_grad = True

        self.encoder.classifier = nn.Identity()


        self.classifier = nn.Sequential(
            nn.Linear(self.embed_dim, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        # # Extract features from ResNet50 and ViT
        logits = self.encoder(x)  # Assuming [batch_size, embed_dim]

        logits = self.classifier(logits)
        
        return logits


# Assuming CIFAR10 - adapt transforms for your dataset
transform = Compose([
    Resize((224,224)),
    RandomHorizontalFlip(),
    # v2.ColorJitter(brightness=(0.5,1.5),contrast=(1),saturation=(0.5,1.5),hue=(-0.1,0.1)),
    ToTensor(),
    Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
])

# Dataset and DataLoader setup
class FTDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

def train_model(train_loader, test_loader):
    accelerator = Accelerator(mixed_precision="fp16")
    model = lcnet_ft()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    loss_fn = nn.CrossEntropyLoss()

    model, optimizer, train_loader, test_loader = accelerator.prepare(model, optimizer, train_loader, test_loader)

    # time start
    
    start = time.time()
    for epoch in range(10):  # Adjust the number of epochs if needed
        model.train()
        total_loss = 0
        for x, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs, labels)
            accelerator.backward(loss)
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")
    # time end
    end = time.time()
    print(f"Training time: {end - start}")

    # Optional test phase at the end of training

    model.eval()
    test_loss = 0
    with torch.no_grad():
        # time start
        start = time.time()
        for x, labels in test_loader:
            outputs = model(x)
            loss = loss_fn(outputs, labels)
            test_loss += loss.item()
        # time end
        end = time.time()
        print(f"Inference time: {(end - start)/len(test_loader)}")
    avg_test_loss = test_loss / len(test_loader)
    print(f"Test Loss: {avg_test_loss:.4f}")

    # Save the final model
    model_path = "/home/chen/EECE570/new_models/single_model/model_lcnet_ft_epoch_10.pt"
    unencapsulated_model = accelerator.unwrap_model(model)
    accelerator.save(unencapsulated_model.state_dict(), model_path)


def main():
    root_dir = './data'
    batch_size = 128
    transform = Compose([
        Resize((224,224)),
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
    ])
    
    # Load full dataset
    full_dataset = CIFAR10(root=root_dir, train=True, download=True, transform=None)
    images = [image for image, _ in full_dataset]
    labels = [label for _, label in full_dataset]

    # Stratified train-test split
    train_images, test_images, train_labels, test_labels = train_test_split(
        images, labels, test_size=0.2, stratify=labels, random_state=42
    )

    # Creating datasets
    train_dataset = FTDataset(train_images, train_labels, transform=transform)
    test_dataset = FTDataset(test_images, test_labels, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

    # Call train_model or any other training function here
    train_model(train_loader, test_loader)

notebook_launcher(main, num_processes=1)

Launching training on one GPU.
Files already downloaded and verified


Epoch 1: 100%|██████████| 313/313 [00:12<00:00, 25.19it/s]


Epoch 1, Loss: 0.6281


Epoch 2: 100%|██████████| 313/313 [00:12<00:00, 25.47it/s]


Epoch 2, Loss: 0.2536


...

Epoch 9: 100%|██████████| 313/313 [00:12<00:00, 25.24it/s]


Epoch 9, Loss: 0.0355


Epoch 10: 100%|██████████| 313/313 [00:12<00:00, 25.59it/s]

Epoch 10, Loss: 0.0287
Training time: 123.08698987960815


Inference time: 0.030752459658852108
Test Loss: 0.2676


In [8]:
# testing on testset
model = lcnet_ft()
model.load_state_dict(torch.load("/home/chen/EECE570/new_models/single_model/model_lcnet_ft_epoch_10.pt"))
model.cuda()

full_dataset = CIFAR10(root='./data', train=False, download=True, transform=None)
images = [image for image, _ in full_dataset]
labels = [label for _, label in full_dataset]


testset = FTDataset(images, labels, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True)


model.eval()
total = 0
correct = 0
false = 0
false_positive = 0
false_negative = 0
with torch.no_grad():
    for x, labels in tqdm(testloader):
        x, labels = x.cuda(), labels.cuda()
        outputs = model(x)
        predictions = torch.argmax(outputs, 1)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()
        false_positive += ((predictions == 1) & (labels == 0)).sum().item()
        false_negative += ((predictions == 0) & (labels == 1)).sum().item()


# output accuracy, recell, precision, f1 score
accuracy = correct / total
recall = correct / (correct + false_negative)
precision = correct / (correct + false_positive)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Accuracy: {:.4f}".format(accuracy))
print("Recall: {:.4f}".format(recall))
print("Precision: {:.4f}".format(precision))
print("F1 Score: {:.4f}".format(f1_score))


Files already downloaded and verified


100%|██████████| 157/157 [00:02<00:00, 59.05it/s]

Accuracy: 0.9318
Recall: 0.9995
Precision: 0.9998
F1 Score: 0.9996


**timm/mnasnet_small.lamb_in1k**

In [9]:

class mnasnet_ft(nn.Module):
    def __init__(self, num_classes=10, embed_dim=1280):
        super(mnasnet_ft, self).__init__()

        self.embed_dim = embed_dim
        
        # Load and freeze lcnet
        self.encoder = timm.create_model('timm/mnasnet_small.lamb_in1k', pretrained=True, num_classes=num_classes)
        # for param in self.encoder.parameters():
        #     param.requires_grad = False

        # unfreeze the last layer
        # for param in self.encoder.classifier.parameters():
        #     param.requires_grad = True

        self.encoder.classifier = nn.Identity()


        self.classifier = nn.Sequential(
            nn.Linear(self.embed_dim, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        # # Extract features from ResNet50 and ViT
        logits = self.encoder(x)  # Assuming [batch_size, embed_dim]

        logits = self.classifier(logits)
        
        return logits


# Assuming CIFAR10 - adapt transforms for your dataset
transform = Compose([
    Resize((224,224)),
    RandomHorizontalFlip(),
    # v2.ColorJitter(brightness=(0.5,1.5),contrast=(1),saturation=(0.5,1.5),hue=(-0.1,0.1)),
    ToTensor(),
    Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
])

# Dataset and DataLoader setup
class FTDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

def train_model(train_loader, test_loader):
    accelerator = Accelerator(mixed_precision="fp16")
    model = mnasnet_ft()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    loss_fn = nn.CrossEntropyLoss()

    model, optimizer, train_loader, test_loader = accelerator.prepare(model, optimizer, train_loader, test_loader)

    # time start
    
    start = time.time()
    for epoch in range(10):  # Adjust the number of epochs if needed
        model.train()
        total_loss = 0
        for x, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs, labels)
            accelerator.backward(loss)
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")
    # time end
    end = time.time()
    print(f"Training time: {end - start}")

    # Optional test phase at the end of training

    model.eval()
    test_loss = 0
    with torch.no_grad():
        # time start
        start = time.time()
        for x, labels in test_loader:
            outputs = model(x)
            loss = loss_fn(outputs, labels)
            test_loss += loss.item()
        # time end
        end = time.time()
        print(f"Inference time: {(end - start)/len(test_loader)}")
    avg_test_loss = test_loss / len(test_loader)
    print(f"Test Loss: {avg_test_loss:.4f}")

    # Save the final model
    model_path = "/home/chen/EECE570/new_models/single_model/model_mnasnet_ft_epoch_10.pt"
    unencapsulated_model = accelerator.unwrap_model(model)
    accelerator.save(unencapsulated_model.state_dict(), model_path)


def main():
    root_dir = './data'
    batch_size = 128
    transform = Compose([
        Resize((224,224)),
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
    ])
    
    # Load full dataset
    full_dataset = CIFAR10(root=root_dir, train=True, download=True, transform=None)
    images = [image for image, _ in full_dataset]
    labels = [label for _, label in full_dataset]

    # Stratified train-test split
    train_images, test_images, train_labels, test_labels = train_test_split(
        images, labels, test_size=0.2, stratify=labels, random_state=42
    )

    # Creating datasets
    train_dataset = FTDataset(train_images, train_labels, transform=transform)
    test_dataset = FTDataset(test_images, test_labels, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

    # Call train_model or any other training function here
    train_model(train_loader, test_loader)

notebook_launcher(main, num_processes=1)

Launching training on one GPU.
Files already downloaded and verified


Epoch 1: 100%|██████████| 313/313 [00:14<00:00, 21.37it/s]


Epoch 1, Loss: 0.5119


Epoch 2: 100%|██████████| 313/313 [00:14<00:00, 21.18it/s]


Epoch 2, Loss: 0.2090


...

Epoch 9: 100%|██████████| 313/313 [00:14<00:00, 21.33it/s]


Epoch 9, Loss: 0.0465


Epoch 10: 100%|██████████| 313/313 [00:14<00:00, 21.40it/s]

Epoch 10, Loss: 0.0367
Training time: 146.61259841918945


Inference time: 0.030660264099700543
Test Loss: 0.2699


In [10]:
# testing on testset
model = mnasnet_ft()
model.load_state_dict(torch.load("/home/chen/EECE570/new_models/single_model/model_mnasnet_ft_epoch_10.pt"))
model.cuda()

full_dataset = CIFAR10(root='./data', train=False, download=True, transform=None)
images = [image for image, _ in full_dataset]
labels = [label for _, label in full_dataset]


testset = FTDataset(images, labels, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True)



model.eval()
total = 0
correct = 0
false = 0
false_positive = 0
false_negative = 0
with torch.no_grad():
    for x, labels in tqdm(testloader):
        x, labels = x.cuda(), labels.cuda()
        outputs = model(x)
        predictions = torch.argmax(outputs, 1)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()
        false_positive += ((predictions == 1) & (labels == 0)).sum().item()
        false_negative += ((predictions == 0) & (labels == 1)).sum().item()


# output accuracy, recell, precision, f1 score
accuracy = correct / total
recall = correct / (correct + false_negative)
precision = correct / (correct + false_positive)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Accuracy: {:.4f}".format(accuracy))
print("Recall: {:.4f}".format(recall))
print("Precision: {:.4f}".format(precision))
print("F1 Score: {:.4f}".format(f1_score))


Files already downloaded and verified


100%|██████████| 157/157 [00:02<00:00, 63.13it/s]

Accuracy: 0.9287
Recall: 0.9997
Precision: 0.9994
F1 Score: 0.9995


***timm/repghostnet_050.in1k***

In [11]:
class repghostnet_ft(nn.Module):
    def __init__(self, num_classes=10, embed_dim=1280):
        super(repghostnet_ft, self).__init__()

        self.embed_dim = embed_dim
        
        # Load and freeze lcnet
        self.encoder = timm.create_model('timm/repghostnet_050.in1k', pretrained=True, num_classes=num_classes)
        # for param in self.encoder.parameters():
        #     param.requires_grad = False

        # unfreeze the last layer
        # for param in self.encoder.classifier.parameters():
        #     param.requires_grad = True

        self.encoder.classifier = nn.Identity()


        self.classifier = nn.Sequential(
            nn.Linear(self.embed_dim, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        # # Extract features from ResNet50 and ViT
        logits = self.encoder(x)  # Assuming [batch_size, embed_dim]

        logits = self.classifier(logits)
        
        return logits


# Assuming CIFAR10 - adapt transforms for your dataset
transform = Compose([
    Resize((224,224)),
    RandomHorizontalFlip(),
    # v2.ColorJitter(brightness=(0.5,1.5),contrast=(1),saturation=(0.5,1.5),hue=(-0.1,0.1)),
    ToTensor(),
    Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
])

# Dataset and DataLoader setup
class FTDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

def train_model(train_loader, test_loader):
    accelerator = Accelerator(mixed_precision="fp16")
    model = repghostnet_ft()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    loss_fn = nn.CrossEntropyLoss()

    model, optimizer, train_loader, test_loader = accelerator.prepare(model, optimizer, train_loader, test_loader)

    # time start
    
    start = time.time()
    for epoch in range(10):  # Adjust the number of epochs if needed
        model.train()
        total_loss = 0
        for x, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs, labels)
            accelerator.backward(loss)
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")
    # time end
    end = time.time()
    print(f"Training time: {end - start}")

    # Optional test phase at the end of training

    model.eval()
    test_loss = 0
    with torch.no_grad():
        # time start
        start = time.time()
        for x, labels in test_loader:
            outputs = model(x)
            loss = loss_fn(outputs, labels)
            test_loss += loss.item()
        # time end
        end = time.time()
        print(f"Inference time: {(end - start)/len(test_loader)}")
    avg_test_loss = test_loss / len(test_loader)
    print(f"Test Loss: {avg_test_loss:.4f}")

    # Save the final model
    model_path = "/home/chen/EECE570/new_models/single_model/model_repghostnet_ft_epoch_10.pt"
    unencapsulated_model = accelerator.unwrap_model(model)
    accelerator.save(unencapsulated_model.state_dict(), model_path)


def main():
    root_dir = './data'
    batch_size = 128
    transform = Compose([
        Resize((224,224)),
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
    ])
    
    # Load full dataset
    full_dataset = CIFAR10(root=root_dir, train=True, download=True, transform=None)
    images = [image for image, _ in full_dataset]
    labels = [label for _, label in full_dataset]

    # Stratified train-test split
    train_images, test_images, train_labels, test_labels = train_test_split(
        images, labels, test_size=0.2, stratify=labels, random_state=42
    )

    # Creating datasets
    train_dataset = FTDataset(train_images, train_labels, transform=transform)
    test_dataset = FTDataset(test_images, test_labels, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

    # Call train_model or any other training function here
    train_model(train_loader, test_loader)

notebook_launcher(main, num_processes=1)

Launching training on one GPU.
Files already downloaded and verified


Epoch 1: 100%|██████████| 313/313 [00:24<00:00, 12.74it/s]


Epoch 1, Loss: 0.6970


Epoch 2: 100%|██████████| 313/313 [00:24<00:00, 12.88it/s]


Epoch 2, Loss: 0.3158


...

Epoch 9: 100%|██████████| 313/313 [00:24<00:00, 12.92it/s]


Epoch 9, Loss: 0.0828


Epoch 10: 100%|██████████| 313/313 [00:24<00:00, 12.98it/s]

Epoch 10, Loss: 0.0715
Training time: 243.02627849578857


Inference time: 0.03105395353293117
Test Loss: 0.2278


In [12]:
# testing on testset
model = repghostnet_ft()
model.load_state_dict(torch.load("/home/chen/EECE570/new_models/single_model/model_repghostnet_ft_epoch_10.pt"))
model.cuda()

full_dataset = CIFAR10(root='./data', train=False, download=True, transform=None)
images = [image for image, _ in full_dataset]
labels = [label for _, label in full_dataset]


testset = FTDataset(images, labels, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True)



model.eval()
total = 0
correct = 0
false = 0
false_positive = 0
false_negative = 0
with torch.no_grad():
    for x, labels in tqdm(testloader):
        x, labels = x.cuda(), labels.cuda()
        outputs = model(x)
        predictions = torch.argmax(outputs, 1)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()
        false_positive += ((predictions == 1) & (labels == 0)).sum().item()
        false_negative += ((predictions == 0) & (labels == 1)).sum().item()


# output accuracy, recell, precision, f1 score
accuracy = correct / total
recall = correct / (correct + false_negative)
precision = correct / (correct + false_positive)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Accuracy: {:.4f}".format(accuracy))
print("Recall: {:.4f}".format(recall))
print("Precision: {:.4f}".format(precision))
print("F1 Score: {:.4f}".format(f1_score))


Files already downloaded and verified


100%|██████████| 157/157 [00:02<00:00, 62.94it/s]

Accuracy: 0.9289
Recall: 0.9991
Precision: 0.9997
F1 Score: 0.9994
